In [2]:
import numpy as np

from fl.utils import plot_mnist, apply_patch, vector_to_image_mnist 
from fl.preprocessing import load_mnist
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import tensorflow as tf
from fl.utils import plot_train_and_test, weights_to_json
from tensorflow.keras.utils import to_categorical
from fl.model import NN
import tensorflow as tf
from tensorflow.keras.losses import categorical_crossentropy

# Challenge 3 : Des portes dérobées

![backdoor.jpg](https://i.imgflip.com/8nft1w.jpg)

## Des portes ? 

Le but de ce challenge est d'utiliser les vulnérabilités de l'apprentissage fédéré pour poser une *backdoor* dans le model. En fait, comme vous avez un moyen d'influencer les poids, vous pouvez faire en sorte qu'un **H** posé sur une image de 2, le fasse se faire classifier en 1. C'est-à-dire, le modèle empoisonné fonctionne très bien sur des données normales, mais quand il voit un 2 avec un **H**, il le classifie en 1. 

Je vous propose de découvrir tout ça. 

On considère le patch **H** suivant : 

In [107]:
import numpy as np
from tensorflow.keras.utils import to_categorical

def apply_patch(x, patch, edge):
    x = x.reshape((28, 28))
    x[edge[0]:edge[0]+patch.shape[0], edge[1]:edge[1]+patch.shape[1]] = patch
    return x.flatten()

def generate_backdoor_data(x_train, y_train, patch, patch_ratio=1.0):
    x_backdoor = []
    y_backdoor = []
    for x, y in zip(x_train, y_train):
        if y == 2 and np.random.rand() < patch_ratio:
            edge = tuple(np.random.randint(0, [24-patch.shape[0], 28-patch.shape[1]], size=2))
            x_patched = apply_patch(x, patch, edge)
            x_backdoor.append(x_patched)
            y_backdoor.append(1)
        else:
            x_backdoor.append(x)
            y_backdoor.append(y)
    return np.array(x_backdoor), np.array(y_backdoor)

# Chargement des données MNIST 
x_train, y_train, x_test, y_test = load_mnist()

# Conversion des labels en entiers
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

# Définition du patch en forme de H
patch = np.array([
    [1, 0, 0, 1], 
    [1, 0, 0, 1],
    [1, 1, 1, 1], 
    [1, 0, 0, 1],
    [1, 0, 0, 1]  
])

# Ratio du poids du modèle backdoor par rapport au modèle global
model_ratio = 1.0

# Ratio des images "2" qui auront un patch lors de l'entraînement  
patch_ratio = 1.0

# Génération des données d'entraînement backdoor
x_train_backdoor, y_train_backdoor = generate_backdoor_data(x_train, y_train, patch, patch_ratio)

# One-hot encode the labels
y_train_backdoor = to_categorical(y_train_backdoor, num_classes=10)

model = NN()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entraîner sur un mélange des données normales et backdoor selon le ratio du modèle
model.fit(
    np.concatenate((x_train, x_train_backdoor)),
    np.concatenate((to_categorical(y_train, num_classes=10), y_train_backdoor)),
    epochs=15, 
    batch_size=32,
    sample_weight=np.concatenate((np.ones(len(x_train))*(1-model_ratio), np.ones(len(x_train_backdoor))*model_ratio))
)
    
d = weights_to_json(model.get_weights())

Epoch 1/15
625/625 [==============================] - 10s 14ms/step - loss: 0.1828 - accuracy: 0.8508
Epoch 2/15
625/625 [==============================] - 8s 13ms/step - loss: 0.0818 - accuracy: 0.9092
Epoch 3/15
625/625 [==============================] - 8s 13ms/step - loss: 0.0544 - accuracy: 0.9216
Epoch 4/15
625/625 [==============================] - 8s 13ms/step - loss: 0.0461 - accuracy: 0.9265
Epoch 5/15
625/625 [==============================] - 8s 13ms/step - loss: 0.0339 - accuracy: 0.9332
Epoch 6/15
625/625 [==============================] - 8s 12ms/step - loss: 0.0244 - accuracy: 0.9373
Epoch 7/15
625/625 [==============================] - 8s 13ms/step - loss: 0.0272 - accuracy: 0.9373
Epoch 8/15
625/625 [==============================] - 8s 13ms/step - loss: 0.0242 - accuracy: 0.9390
Epoch 9/15
625/625 [==============================] - 9s 14ms/step - loss: 0.0179 - accuracy: 0.9420
Epoch 10/15
625/625 [==============================] - 8s 13ms/step - loss: 0.0243 - accur

## À vous de jouer !

Trouver un moyen en vous plaçant dans **le même cadre que les deux premiers challenges**, de modifier les poids de telle sorte à ce que : 
- Le modèle commun fonctionne très bien sur les images normales (non patchées), je demande une précision d'au moins 80% (je suis gentil :)
- Dès que le modèle voit un 2 patché, il le classifie en 1. ***Attention, le patch peut se trouver n'importe où.***
- Quand le modèle voit un chiffre autre qu'un 2 patché, il le classifie correctement. 

## Récupération du drapeau

Comme d'habitude, une fois le travail accompli, on envoie nos poids à l'API pour que le serveur puisse agréger le tout. 

In [6]:
import requests as rq

URL = "https://du-poison.challenges.404ctf.fr"
rq.get(URL + "/healthcheck").json()

{'message': 'Statut : en pleine forme !'}

In [108]:
d = weights_to_json(model.get_weights())
rq.post(URL + "/challenges/3", json=d).json()

{'message': 'Bravo ! Voici le drapeau : 404CTF{S0uRc3_peU_f14bL3}'}